In [ ]:
from generate_3_sat_constraints import solve_with_solver, generate_3_sat_constraints
from real_constraint_utils import solve_smt_formula, cnf_to_smt_over_reals
from pysat.formula import CNF
import pandas as pd
from tqdm import tqdm
from pysat.solvers import Minisat22, Glucose3, Lingeling

# Generate CNF

In [ ]:
generate_3_sat_constraints(10, formula_dir="/Users/mihajlobulesnij/Documents/system/RAI/project/DRL_DGM/constraints_generated/sat_constraints")

# Convert to constraints over Reals

In [ ]:
dataset = pd.read_csv("constraints_generated/sat_constraints/sat_formulas.csv")
for filename in tqdm(dataset["filename"]):
    time_mesured_buf = []
    cnf = CNF(from_file=f"constraints_generated/sat_constraints/{filename}")
    cnf_to_smt_over_reals(cnf, filename = "constraints_generated/smt_constraints_random/"+filename[:-4], random_v = True)

Generate data that satisfy constraints with DRL

In [ ]:
dataset = pd.read_csv("constraints_generated/sat_constraints/sat_formulas.csv")
for filename in tqdm(dataset["filename"]):
    time_mesured_buf = []
    cnf = CNF(from_file=f"constraints_generated/sat_constraints/{filename}")
    cnf_to_smt_over_reals(cnf, filename = "constraints_generated/smt_constraints/"+filename[:-4], random_v = False)

# Generating Dataset that satisfy constraints

In [ ]:
import os
import subprocess
import pandas as pd
from tqdm import tqdm
base_path = "/Users/mihajlobulesnij/Documents/system/RAI/project/DRL_DGM/constraints_generated/smt_constraints_random"
seeds = [1, 2, 3]
python_script = "sample_random_data.py"

info_cnf = pd.read_csv("/Users/mihajlobulesnij/Documents/system/RAI/project/DRL_DGM/constraints_generated/sat_constraints/sat_formulas.csv")
sats = info_cnf.Sat
filenames = info_cnf.filename
commands = []

for file_name, sat in zip(filenames, sats):
    if sat:
        full_constraint_path = os.path.join(base_path, file_name[:-4]+".txt")
        
        for seed in seeds:
            cmd = f'/opt/homebrew/Caskroom/miniconda/base/envs/drl_env/bin/python {python_script} --seed {seed} --constraint_path "{full_constraint_path}" --save_path data_generated/data_smt_random'
            commands.append(cmd)

In [ ]:
for i, cmd in tqdm(enumerate(commands, 1)):
    print(f"\n▶️ Running command {i}/{len(commands)}:\n{cmd}\n")
    
    result = subprocess.run(cmd, shell=True)

    if result.returncode != 0:
        print(f"❌ Command failed with exit code {result.returncode}:")
        print(cmd)
        break
    else:
        print(f"✅ Completed {i}/{len(commands)}\n")

### Training

In [ ]:
import os
import subprocess
import pandas as pd
from tqdm import tqdm
base_path = "constraints_generated/smt_constraints_random"
python_script = "main.py"

seeds = [1, 2]

info_cnf = pd.read_csv("constraints_generated/sat_constraints/sat_formulas.csv")
sats = info_cnf.Sat
filenames = info_cnf.filename
commands = []

for file_name, sat in zip(filenames, sats):
    if sat:
        full_constraint_path = os.path.join(base_path, file_name[:-4]+".txt")
        for seed in seeds:
            cmd = f'/opt/homebrew/Caskroom/miniconda/base/envs/drl_env/bin/python {python_script} --seed {42} --constraint_path "{full_constraint_path}" --dataset_index "{seed}"'
            commands.append(cmd)

In [ ]:
for i, cmd in tqdm(enumerate(commands[100:], 1)):
    print(f"\n▶️ Running command {i}/{len(commands)}:\n{cmd}\n")
    
    result = subprocess.run(cmd, shell=True)

    if result.returncode != 0:
        print(f"❌ Command failed with exit code {result.returncode}:")
        print(cmd)
        break
    else:
        print(f"✅ Completed {i}/{len(commands)}\n")
